# Formulation avec cycles de taille 2

In [ ]:
function get_cycles_2(graph)
    """
    Construit les données utiles du graphe restreint aux cycles de taille 2.
    
    Entrée : 
        - graph : array - Liste des caractéristiques du graphe considéré
    
    Sortie :
        - C_2          : array - Liste des identifiants des cycles de taille 2
        - collect(V_2) : array - Liste des sommets concernés par les cycles de taille 2
        - C_v          : array - Liste des identifiants des cycles passant par chaque sommet 
        - A_2          : array - Liste des arcs concernés par les cycles de taille 2
    """
    # Récupération des données du graphe
    n = graph[1]
    A = graph[2]
    preds = graph[4]
    succ = graph[5]
    
    # Initialisation des listes de cycles, sommets, et cycles passant par chaque sommet  
    C_2 = []                          # chaque cycle est représenté par un des deux arcs qui le compose
    V_2 = Set()                       # liste des sommets du graphe restreint aux cycles de taille 2
    C_v = [[] for i in Array(1:n)]    # liste des cycles passant par chaque sommet du graphe
    A_2 = []                          # liste des arcs du graphe restreint aux cycles de taille 2
    
    # Mise a jour itérative des listes
    for i in Array(1:n)
        for j in succ[i]
            # On ne considère que la moitié des arcs
            if i > j
                continue
            end
            # Si on a un cycle possible
            if (j,i) in A 
                # Ajout du cycle dans la liste C_2
                push!(C_2,(i,j))
                    
                push!(A_2,(i,j))
                push!(A_2,(j,i))
                        
                # Ajout du cycle dans les ensembles C_v[i] et C_v[j]
                push!(C_v[i], (i,j))
                push!(C_v[j], (i,j))
                        
                # Ajout des sommets dans la liste des sommets "intéressants"
                push!(V_2, i)
                push!(V_2, j)
                
            end  
        end
    end
        
    return C_2,collect(V_2),C_v, A_2
end

In [1]:
function uncertain_model_2(C_2,V_2,C_v, S, B, logs, relax=false, t_max=120)
    """
    Crée et optimise le programme linéaire avec cycles de taille 2.

    Entrées : 
        - C_2   : array   - Liste des identifiants des cycles de taille 2
        - V_2   : array   - Liste des sommets concernés par les cycles de taille 2
        - C_v   : array   - Liste des identifiants des cycles passant par chaque sommet 
        - S     : array   - Liste des scénarios
        - B     : Integer - Budget (nombre de tests maximum)
        - logs  : Boolean - Booléen pour l'affichage de la solution
        - relax : Boolean - Booléen pour relâcher la contrainte d'intégrité
        - t_max : Integer - Temps maximum accordé pour la résolution du problème
    
    Sortie : 
        - objective_value(model) : Float      - Valeur optimale du modèle résolu 
        - model                  : JuMP Model - Modèle du problème
        - timer                  : Float      - Temps de calcul
    """
    # Vecteur des sommets 
    N = Array(1:length(S))
    
    # Création du modèle
    model = Model(HiGHS.Optimizer); 
    set_optimizer_attribute(model,"output_flag",false)

    # Variables
    ## x[(i,j)] = 1 ssi on réalise un test croisé sur les deux arcs (i,j) et (j,i)    
    @variable(model, x[C_2], Bin);
    ## z[(i,j),s] = 1 ssi le cycle [(i,j),(j,i)] effectue un transfert dans le scénario s
    if relax
        @variable(model, z[C_2,N] >=0);
    else
        @variable(model, z[C_2,N], Bin);
    end
    
    # Fonction objectif: On souhaite maximiser la moyenne empirique du nombre de transplantations
    @objective(model, Max, 2/length(S)*sum(z[c,s] for c in C_2 for s in N));
    
    # Contraintes
    ## On réalise au plus B tests croisés (B = budget de tests)
    @constraint(model,c1, 2*sum(x[a] for a in C_2) <= B)
        
    ## On ne peut choisir un arc que si le test croisé a été réalisé et a été réussi pour cet arc (dans le scénario s)
    @constraint(model,c2[c in C_2, s in N],
        2 * z[ (c[1],c[2]) , s ] <= x[c] * ( S[s][c[2],c[1]] + S[s][c[1],c[2]]) )
    
    ## Chaque sommet peut être parcouru par au plus un cycle, dans le scénario s
    @constraint(model,c4[v in V_2, s in N], sum(z[c,s] for c in C_v[v]) <= 1)
    
    # Ajout du time_limit souhaité
    set_time_limit_sec(model, t_max)
    
    timer = @timed optimize!(model);  
    
    # Affichage si time_limit atteint
    if termination_status(model) == TIME_LIMIT
        println("Attention : time_limit atteint")
    end
    # Affichage de la solution si souhaité
    if logs 
        println("***** Uncertain *****");
        # Statistiques de la solution
        println("Statut de la solution = ", termination_status(model));
        println("Valeur optimale = ", objective_value(model));
        println("Temps = ", timer[2])
    end
            
    return objective_value(model),model,timer;
end

LoadError: LoadError: UndefVarError: `@variable` not defined
in expression starting at In[1]:34

In [ ]:
function pb_recours_2(C_2,V_2,C_v, S, logs, X, relax=false)
    """
    Résout le programme linéaire avec cycles de taille 2 pour un ensemble de tests "X" donné.

    Entrées : 
        - C_2   : array          - Liste des identifiants des cycles de taille 2
        - V_2   : array          - Liste des sommets concernés par les cycles de taille 2
        - C_v   : array          - Liste des identifiants des cycles passant par chaque sommet 
        - S     : array          - Liste des scénarios
        - logs  : Boolean        - Booléen pour l'affichage de la solution
        - X     : DenseAxisArray - Liste des tests croisés à réaliser
        - relax : Boolean        - Booléen pour relâcher la contrainte d'intégrité
    
    Sortie : 
        - objective_value(model) : Float      - Valeur optimale du modèle résolu 
        - model                  : JuMP Model - Modèle du problème
        - timer                  : Float      - Temps de calcul
    """
    # Vecteur des sommets 
    N = Array(1:length(S))

    # Création du modèle
    model = Model(HiGHS.Optimizer); 
    set_optimizer_attribute(model,"output_flag",false)

    # Variables
    ## z[(i,j),s] = 1 ssi le cycle [(i,j),(j,i)] effectue un transfert dans le scénario s
    @variable(model, z[C_2,N], Bin);
    
    # Fonction objectif: On souhaite maximiser la moyenne empirique du nombre de transplantations
    @objective(model, Max, 2/length(S)*sum(z[c,s] for c in C_2 for s in N));
    
    # Contraintes
    ## On ne peut choisir un arc que si le test croisé a été réalisé et a été réussi pour cet arc (dans le scénario s)
    @constraint(model,c2[c in C_2, s in N],
        2 * z[ (c[1],c[2]) , s ] <= X[c] * ( S[s][c[2],c[1]] + S[s][c[1],c[2]]) )
    
    ## Chaque sommet peut être parcouru par au plus un cycle, dans le scénario s
    @constraint(model,c4[v in V_2, s in N], sum(z[c,s] for c in C_v[v]) <= 1)
    
    if relax
        # Si l'on souhaite résoudre le problème avec contrainte d'intégrité
        relax_integrality(model)
    end 
    
    timer = @timed optimize!(model);  
    # Affichage de la solution si souhaité
    if logs 
        println("***** Uncertain *****");
        # Statistiques de la solution
        println("Statut de la solution = ", termination_status(model));
        println("Valeur optimale = ", objective_value(model));
        println("Temps = ", timer[2])
    end
            
    return objective_value(model),model,timer;
end

In [ ]:
function compute_solution_uncertain_2(model)
    """
    Crée et affiche les cycles solutions rendus par le modèle fourni.
    
    Entrées : 
        - model : JuMP Model - Modèle résolu du problème
    
    Sorties : 
        - arcs : Array - Cycles solutions
    """
    # Récupération des valeurs optimales des variables et de la valeur optimale
    X = value.(model[:x])

    # Récupération des arcs solution
    arcs = []
    for (i,j) in axes(X)[1]
        if abs(X[(i,j)]-1) < 1e-6
            push!(arcs,(i,j))
        end
    end
        
    println("Tests croisés à réaliser : ")
    test = true
    while length(arcs) > 0
        (i,j) = arcs[1]
        print("  - ")
        print((i,j)) 
        print(" & ")
        println((j,i))

        arcs = setdiff(arcs, [(i,j),(j,i)])
    end
    
    return arcs
end

In [ ]:
println("Fonctions de résolution du problème de taille 2 importées avec succès.")